<a href="https://colab.research.google.com/github/Mezzzer/Snorkel-labeling/blob/main/Snorkel_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJEKT NUM - SNORKEL

## Instalacja bibliotek

In [ ]:
!pip install snorkel
!pip install spacy_langdetect 
!pip install spacy
!pip install spacytextblob
!python -m spacy download en_core_web_sm
# !python -m textblob.download_corpora
# !pip install spacytextblob==0.1.7
# !pip install spacytextblob -U
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm
!pip install spacy-transformers

## Import bibliotek i podpięcie dysku

In [87]:
import pandas as pd
import numpy as np
import re 
from snorkel.labeling import labeling_function
from textblob import TextBlob
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from spacytextblob.spacytextblob import SpacyTextBlob
from textblob import TextBlob
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preprocessing (można pominąć i przejść do "Tworzenia modelu Snorkela")

### Wczytanie datasetu

In [77]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

dataset = pd.read_csv('drive/MyDrive/NUM/steam_reviews.csv')
dataset['label'] = np.where(dataset['recommendation'] == 'Recommended', 1, 0)
dataset = reviews_data[['review','label']]
reviews = dataset[['review','label']]
reviews = reviews.rename(columns={'review': 'text'})
reviews['text'] = reviews['text'].astype(str)
reviews.head()

,text,label
0,&gt Played as German Reich&gt Declare war on B...,1
1,yes.,1
2,Very good game although a bit overpriced in my...,1
3,Out of all the reviews I wrote This one is pro...,1
4,Disclaimer I survivor main. I play games for f...,1


### Usunięcie recenzji napisanych w innym języku niż angielski oraz brakujących wartości

In [81]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

reviews = reviews.dropna()
reviews = reviews.drop(reviews[reviews['text'].map(len) < 3].index)
reviews_part1 = reviews.iloc[:200000].copy()
reviews_part1['language'] = reviews_part1['text'].apply(lambda rev: nlp(rev)._.language['language'])

In [82]:
english_reviews = reviews_part1[reviews_part1['language']=='en'][['text','label']]
english_reviews.to_csv("drive/MyDrive/NUM/reviews_english.csv",index=False)

In [17]:
english_reviews.shape

(82499, 2)

## Tworzenie modelu Snorkela

### Wczytanie zapisanych recenzji napisanych po angielsku

In [114]:
reviews_dataset = pd.read_csv('drive/MyDrive/NUM/reviews_english.csv')
reviews_dataset

,text,label
0,&gt Played as German Reich&gt Declare war on B...,1
1,Very good game although a bit overpriced in my...,1
2,Out of all the reviews I wrote This one is pro...,1
3,Disclaimer I survivor main. I play games for f...,1
4,ENGLISH After playing for more than two years ...,1
...,...,...
163874,Product received for free. GTA V is one of the...,1
163875,Don't get me wrong this is a great game online...,0
163876,GTA 5 is a great game. The people who make it?...,1
163877,EDIT 7/10/17 OpenIV is back it'll be interesti...,0


### Podział na zbiór uczący i testowy

In [91]:
X_train, X_test = train_test_split(reviews_dataset, test_size=0.2, random_state=0)
X_test

,text,label
139642,GTA Online doesn't work. Never gonna pay for t...,0
99134,it's like spend your money simulator but u don...,0
93332,This was extremely fun until they screwed with...,0
106323,When you attack modders you are basically aski...,0
122755,this game woth to buy than Battlefield Hardlin...,1
...,...,...
43086,It's like soccer... BUT BALLLLLIN!!!! WITH ACT...,1
81675,Good game..easy to learn how to play..my sons ...,1
51962,I don't know why this game is popular especial...,0
62458,it is one of the best games i have ever played...,1


In [92]:
X_test.to_csv("drive/MyDrive/NUM/test_dataset.csv",index=False)

### Labeling functions

In [96]:
reviews = X_train.copy()
reviews['label'] = -1
reviews

,text,label
130751,great game,-1
104827,Was really dissapointed at the fact I couldnt ...,-1
32847,It's pretty fun,-1
36612,Best game ever even if you don't like cars or ...,-1
67605,this game is the besttttt i really like it,-1
...,...,...
97639,DUMBASS Take Two KILLED Open IV because they'r...,-1
95939,For anyone buying this game and looking to pla...,-1
152315,Don't bother with companies that kill mod support,-1
117952,Im to lazy to right but I just saw a modder in...,-1


In [97]:
@labeling_function()
def good(review):
    if 'not good' in review.text.lower():
        return POSITIVE
    elif 'good' in review.text.lower():
        return NEGATIVE
    else:
        return ABSTAIN

@labeling_function()
def bad(review):
    if 'not bad' in review.text.lower():
        return POSITIVE
    elif 'bad' in review.text.lower():
        return NEGATIVE
    else:
        return ABSTAIN

In [98]:
@labeling_function()
def love_it(review):
  return POSITIVE if 'love it' in review.text.lower() else ABSTAIN

@labeling_function()
def hate_it(review):
  return POSITIVE if 'love it' in review.text.lower() else ABSTAIN

@labeling_function()
def like_it(review):
    if 'don\'t like it' in review.text.lower():
        return NEGATIVE
    elif 'like it' in review.text.lower():
        return POSITIVE
    else:
        return ABSTAIN

In [99]:
@labeling_function()
def positive_polarity(review):
    return POSITIVE if TextBlob(review.text.lower()).sentiment_assessments.polarity > 0.3 else ABSTAIN

@labeling_function()
def negative_polarity(review):
    return NEGATIVE if TextBlob(review.text.lower()).sentiment_assessments.polarity < -0.3 else ABSTAIN

@labeling_function()
def recommend(review):
    if 'not recommend' in review.text.lower():
        return NEGATIVE
    elif 'recommend' in review.text.lower():
        return POSITIVE
    else:
        return ABSTAIN

### Zaplikowanie funkcji

In [100]:
from snorkel.labeling import PandasLFApplier

lfs = [good, bad, like_it, love_it, hate_it, positive_polarity, negative_polarity, recommend]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=reviews)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 131103/131103 [04:04<00:00, 535.82it/s]


### Pokrycie funkcji

In [101]:
coverage_good, coverage_bad, coverage_like, coverage_love,coverage_hate, coverage_posi, coverage_negat, coverage_recom = (L_train != ABSTAIN).mean(axis=0)

print(f"Pokrycie dla funkcji good(): {coverage_good * 100:.1f}%")
print(f"Pokrycie dla funkcji bad(): {coverage_bad * 100:.1f}%")
print(f"Pokrycie dla funkcji like_it(): {coverage_like * 100:.1f}%")
print(f"Pokrycie dla funkcji love_it(): {coverage_love * 100:.1f}%")
print(f"Pokrycie dla funkcji hate_it(): {coverage_hate * 100:.1f}%")
print(f"Pokrycie dla funkcji positive_polarity(): {coverage_posi * 100:.1f}%")
print(f"Pokrycie dla funkcji negative_polarity(): {coverage_negat * 100:.1f}%")
print(f"Pokrycie dla funkcji recommend(): {coverage_recom * 100:.1f}%")

Pokrycie dla funkcji good(): 16.6%
Pokrycie dla funkcji bad(): 5.1%
Pokrycie dla funkcji like_it(): 1.2%
Pokrycie dla funkcji love_it(): 1.5%
Pokrycie dla funkcji hate_it(): 1.5%
Pokrycie dla funkcji positive_polarity(): 15.3%
Pokrycie dla funkcji negative_polarity(): 7.6%
Pokrycie dla funkcji recommend(): 7.8%


In [102]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
good,0,"[0, 1]",0.165511,0.073538,0.057016
bad,1,"[0, 1]",0.050952,0.030396,0.009626
like_it,2,"[0, 1]",0.011647,0.006842,0.005065
love_it,3,[1],0.014576,0.014576,0.003181
hate_it,4,[1],0.014576,0.014576,0.003181
positive_polarity,5,[1],0.152872,0.044522,0.035865
negative_polarity,6,[0],0.075803,0.009824,0.002570
recommend,7,"[0, 1]",0.078412,0.031899,0.023073


### Przykładowy wynik funcji *like_it*

In [103]:
from snorkel.analysis import get_label_buckets
reviews.iloc[L_train[:,2] == POSITIVE]

,text,label
67605,this game is the besttttt i really like it,-1
123884,I will divide this game in 2 revies GTAV and G...,-1
64452,This is probably one fo the most unique and fu...,-1
148919,Great port onto PC of a cult game. Very well o...,-1
123201,Amazing it outshines Gta 4 by a landsllde. you...,-1
...,...,...
28039,I have been playing monster hunter for a long ...,-1
132812,Full of cheatersLoading times are redicilous l...,-1
15741,Good game i like it how it is because its like...,-1
35489,Terraria is quite the fun sandbox game and in ...,-1


### Model

In [104]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)
preds_train = label_model.predict(L=L_train)

In [105]:
import numpy as np

labels, counts = np.unique(preds_train, return_counts=True)

for l, c in zip(labels, counts):
    print(f"LABEL: {l}, count: {c}")

LABEL: -1, count: 73174
LABEL: 0, count: 36718
LABEL: 1, count: 21211


## Ostateczne zbiory uczące



### Ze Snorkelem

In [106]:
reviews['label'] = preds_train
reviews_train = reviews[reviews['label']!=-1][['text','label']]
reviews_train.head()

,text,label
67605,this game is the besttttt i really like it,0
117413,I must preface that this is not a review base ...,0
93684,A game that has killed what made it's previous...,0
154959,Wow This Game Is So GoodI play With My Friend ...,0
133163,Ok this is GTA V it's pretty dank. run aorund ...,1


In [107]:
print(reviews_train.shape)
reviews_train.to_csv("drive/MyDrive/NUM/train_set_snorkel_labels.csv",index=False)

(57929, 2)


### Bez Snorkela

In [111]:
revievs_train_nosno = X_train[reviews['label']!=-1]
revievs_train_nosno

,text,label
67605,this game is the besttttt i really like it,1
117413,I must preface that this is not a review base ...,0
93684,A game that has killed what made it's previous...,0
154959,Wow This Game Is So GoodI play With My Friend ...,1
133163,Ok this is GTA V it's pretty dank. run aorund ...,1
...,...,...
80186,1v1'ing your friends at rocket league gets sur...,1
112420,i hate it i got banned for calling some one fa...,0
41993,Where to Start?I'm not normally a fan of these...,1
97639,DUMBASS Take Two KILLED Open IV because they'r...,0


In [112]:
revievs_train_nosno.to_csv("drive/MyDrive/NUM/train_set_original_labels.csv",index=False)